# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.17.2 OpenAI Embedding模型初级应用

&emsp;&emsp;在快速建立了对Embedding系列技术的基本认知、以及掌握了OpenAI Embedding模型API的调用方法之后，接下来我们尝试将Embedding技术应用到一些实际开发场景中，探索Embedding在大模型技术开发领域的实际用途，并在这个过程中逐渐深入了解OpenAI Embedding模型性能。

&emsp;&emsp;本节我们将先从一个相对简单的场景入手使用Embedding模型——先围绕一组带有标签文本数据集尝试进行Embedding编码，并围绕此项量化之后的结果进行分析和建模，以此探究Embedding在语义分类问题中的表现，并介绍语义分类在大模型开发中的实际应用；而下一小节中我们将进一步介绍更加复杂的开发场景下的Embedding技术应用。

- 语义分类与用户意图挖掘

&emsp;&emsp;所谓语义分类问题，按照大模型领域的专业术语来说，就是根据用户语义判断用户背后意图。本节将采用一个Kaggle竞赛数据集：亚马逊精选美食评论数据集进行语义分类应用方法介绍。该数据集是一个带标签的语义分类数据集，该数据集包含了用户对亚马逊部分商品（美食）的评价，包括文本评价和评分，很明显，评分就是用户意图的量化表示，也就是文本评价的“标签”。在传统NLP领域，该数据集是情感分析的经典数据集，而在大模型技术领域，我们尝试借助该数据集来介绍用户意图挖掘的一种方法，并将其用于提高Agent运行稳定性。

&emsp;&emsp;而什么是用户意图挖掘呢？在此前的课程中我们曾多次介绍到，用户意图对齐是大模型普适性的最根本保证，在Agent开发过程中尤为重要。例如在此前介绍的MateGen开发过程中，能否正确识别用户意图会直接复杂任务拆解的准确性、调用外部函数的准确性等环节。当然，Agent开发的各环节中，模型语义分类性能的提升，最直接的效果就是能够大幅提升Function calling的准确性。

- 语义分类性能提升与Function calling稳定性

&emsp;&emsp;毫无疑问，Function calling的诞生大幅加快了Agent开发效率，使得大模型不再是一个单纯的知识渊博的对话机器人，而是可以调用各类工具帮我们切实完成一些工作的Agent。而Function calling到底如何实现，OpenAI从未公开其背后的技术细节。然而经过长期实践我们发现，Function calling运行的稳定性和大模型本身的性能直接相关。例如GPT-4的Function calling稳定性就要强于GPT-3.5，GPT-4能够在大量外部函数中精确识别满足当前用户需求所需要的外部函数，并且能够顺利识别外部函数参数并进行参数便携，相比之下GPT-3.5性能要弱一些，面对一些功能相似的外部函数往往无法进行有效分辨，例如对于本地代码解释器Python函数和Python绘图函数。而如果是对于类似谷歌Gemini Pro模型（该模型也提供了Function calling功能），则对于大量复杂外部函数进行有效调用都难以做到，而如果是对于目前国内开源大模型，例如ChatGLM3，则围绕单独一个外部函数都无法做到每次“有求必应”。

&emsp;&emsp;而当模型本身自带的Function calling无法满足使用需求（但Function calling又是构建Agent必不可少的功能）时，我们就需要考虑尝试一些其他方法来提高Fucntion calling稳定性，其中最核心的思路就是能否将此前完全在大模型内部自动完成的外部函数选取工作来手动完成，即能不能通过一些其他方法来先确定当前需求要使用的外部函数，进而手动创建Function call message和Function response message。很明显，如果这个手动执行的方法得到的Function calling调用准确性高于模型全自动调用Function calling的准确性，那么我们会更加倾向于手动来完成这个Function calling的外部函数选取+外部函数计算流程。不过很明显的问题是，如果不依靠大模型来判断“哪个问题要调用哪个外部函数”，又有什么方法能够稳定实现这一过程呢？

- 借助Embedding提升手动Function calling稳定性

&emsp;&emsp;在不考虑其他复杂流程和方法的前提下，借助Embedding来完成手动Function calling，是最为高效实用的方法。总的来看，借助Embedding来实现手动Function calling有以下两个核心思路：其一是借助Embedding模型根据词义进行编码的特性，进行零样本分类。即直接根据用户需求和外部函数的函数说明相似度，判断是否需要进行外部函数调用：

In [2]:
import numpy as np
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [14]:
sql_inter_description = "用于执行一段SQL代码，并最终获取telco_db数据库数据查询结果，\
核心功能是将输入的SQL代码传输至MySQL环境中进行运行，\
并最终返回SQL代码运行结果。需要注意的是，本函数是借助pymysql来连接MySQL数据库。"

In [15]:
q1 = "请帮我下telco_db数据库中所有用户的性别和年龄信息。"

In [16]:
q2 = "请帮我介绍下什么是机器学习？"

In [17]:
text_tuples = (sql_inter_description, q1, q2)
res = openai.Embedding.create(
  model="text-embedding-ada-002",
  input=text_tuples,
  encoding_format="float"
)

In [18]:
cosine_similarity([res.data[0].embedding, res.data[1].embedding, res.data[2].embedding])

array([[1.        , 0.80584902, 0.71179257],
       [0.80584902, 1.        , 0.75902879],
       [0.71179257, 0.75902879, 1.        ]])

能够发现，对于基于词义进行Embedding的模型而言，词向量编码结果本身就能一定程度反应用户问题和外部函数调用之间的关联度。当然，基于零样本分类来判断到底需要调用哪个外部函数，还需要一些额外的辅助手段，例如需要海量的用户调用某函数需求来判断相似度基准值，或者通过聚类的方法先将用户需求进行分类并将不同大类对应调用不同类型外部函数，然后再根据编码结果判断当前需求应该属于哪类，再据此判断所需调用的外部函数。

&emsp;&emsp;而第二种思路则是进行用户需求的有监督学习和分类。具体实现流程为先积累大量用户需求文本（即用户问题），然后手动对其进行标注，需要注明在当前用户问题需要调用哪个外部函数（或者无需调用外部函数）来进行回答。考虑到对于大多数Agent来说外部函数不会轻易发生变化（Agent功能不会轻易变动），因此是有机会能够积累到足够大体量的用户需求数据的。而当我们完成用户需求文本数据标注之后，接下来即可对其进行Embedding编码，并且对于新需求也可以实时进行编码，并将编码结果视作数值型特性并进行机器学习建模预测，预测当前用户需求属于哪一类需求，并由此判断回答当前用户问题需要调用哪个外部函数。先进行Embedding、再进行机器学习建模，这类方法也是Word2Vec这类基于语义的Embedding方法诞生之后被广泛尝试行之有效的文本分类方法。

> 关于如何创建用户需求数据集，也可以考虑采用“高性能模型为低性能模型创建数据集”的思路，即例如根据GPT-4模型的Function calling调用结果来作为当前用户需求的标签。

> 当然，除了Embedding方法可以提升模型Function calling性能之外，微调也能在这个过程中发挥作用。相关方法我们将在之后的课程中进行介绍。

- 借助亚马逊精选美食评论数据集实现上述过程

&emsp;&emsp;考虑到目前MateGen项目正在上线测试阶段，目前并未积累足够量的用户数据集，因此课程里面将借助Amazon Fine Food Reviews（亚马逊精选美食评论）数据集，该数据集是一个广泛用于自然语言处理和机器学习研究的公开数据集。它包含了亚马逊网站上超过50万条关于食品的客户评论，这些评论涵盖了1999年至2012年间的用户反馈。每条评论包括多个信息维度，如评论者ID、产品ID、评分（星级）、评论文本、有用性评价（即其他用户对评论有用性的投票），以及评论的时间戳。同时该数据集是一个带有标签的文本数据集，同时也是OpenAI官方教学（OpenAI cookbook）推荐使用的数据集。我们将首先尝试借助该数据集进行用户评价的分类预测、聚类、零样本分类等工作，并尝试将相关方法迁移至Agent开发领域中。

> 关于OpenAI大模型Function calling实现方法，尽管OpenAI一直未公布具体技术实现流程，但外界普遍猜测是采用了单独的意图识别大模型+Embedding进行辅助判断来完成的，实际上这也是目前大模型业内对于意图识别问题的最为有效的处理方法。在后续的进阶模块中，我们也将进一步介绍该方法。

> 关于OpenAI系列模型是否需要进一步提升Function calling功能稳定性，尽管GPT-4-turbo模型进一步提升了复杂外部函数的Function calling稳定性，同时GPT-4-turbo也是目前Function calling性能最好的模型没有之一，但对于部分功能非常相似的外部函数来说，GPT-r-turbo在调用过程中也会出现误判，此时也可以考虑借助本节介绍的方法来提升Function calling的准确性。

### 三、Amazon Fine Food Reviews数据集准备与Embedding过程

- 数据集简介

&emsp;&emsp;Amazon Fine Food Reviews（亚马逊精选美食评论）数据集**，此数据集包含从1999年~2012年10月时间范围内的用户评论，共计568,454条，我们会**从中提取1,000或2000等不同大小的小样本数据集，使用OpenAI 的 Embedding 第二代模型：`text-embedding-ada-002` 模型对抽取出来的小样本数据集中评论文本进行Embedding，将其应用于各个案例中**。

&emsp;&emsp;我们选择使用的数据集是来自Kaggle平台的 `Amazon Fine Food Reviews`（亚马逊精选美食评论）数据集，此数据集包含了从1999年~2012年10月时间范围内的用户评论，共计568,454条，大家可以直接从Kaggle平台将该数据集下载到本地：

> Amazon Fine Food Reviews 数据集下载链接：https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews/

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231102174129099.png" width='800'></div> 

&emsp;&emsp;亚马逊精选食品评论数据集是一个公开的数据集，其数据量大、时间跨度长，且详细记录了各种产品的用户反馈，因为是真实用户生成的内容，这些评论在自然语言的多样性、情感表达的深度以及日常表述的真实性方面都提供了极为丰富的信息，在自然语言处理研究领域（NLP）是非常有价值的数据资源。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231102180405697.png"></div> 

&emsp;&emsp;如上所示，完整的数据集中共有10个特征字段, 非常细致的记录了用户的评价行为和产品的接受度，除了基本的用户和产品信息，还有用户互动和反馈的相关内容，例如该评价对其他人是否有用、评价者的个人感受等等。具体的特征字段解释如下：

| 字段名               | 中文释义             | 描述                                                  |
|:-----------------------|:--------------------|:-----------------------------------------------------|
| Id                    | 行标识符             |                                                      |
| ProductId             | 产品标识符           | 产品的唯一识别码                                          |
| UserId                | 用户标识符           | 用户的唯一识别码                                          |
| ProfileName           | 用户昵称             | 用户的个人昵称                                            |
| HelpfulnessNumerator  | 有用的正面评价数       | 认为该评论有帮助的用户数量                                      |
| HelpfulnessDenominator| 有用评价的总数        | 表示有多少用户表示该评论有帮助或无帮助                                |
| Score                 | 评分                | 产品的评分，介于1到5之间                                       |
| Time                  | 评论时间             | 评论发表的时间戳                                           |
| Summary               | 评论摘要             | 对评论内容的简短总结                                         |
| Text                  | 评论文本             | 用户对产品的具体评价内容                                       |


&emsp;&emsp;正是因为该数据集的多样性和丰富性，使其成为了研究各种NLP任务的理想选择，利用该数据集的不同信息组合，可以构建多种自然语言处理的应用场景，比如：
- **情感分析**：利用评论文本（Text）对情感进行分类，判断用户的情绪是积极的、消极的还是中性的。通过将评分（Score）作为情感强度的标签，可以训练一个模型来识别评论中的情感色彩；
- **用户行为分析**：分析不同用户（UserId）和用户昵称（ProfileName）的用户行为模式，例如，哪些用户更倾向于给出高评分或低评分，或者哪些用户更活跃在评论区；
- **有用性评价预测**：结合有用性投票（HelpfulnessNumerator和HelpfulnessDenominator）来预测评论的有用性。理解什么样的评论内容更可能被视为有用，指导用户如何撰写更具帮助性的评论；
- **推荐系统**：基于用户给出的评分（Score）以及他们对产品的评价（Text），开发推荐算法，为用户推荐他们可能喜欢的其他产品；
- **文本摘要与关键词提取**：使用评论的摘要（Summary）和评论文本（Text）字段来训练模型进行自动文本摘要和关键词提取，快速把握评论的主要内容；
- **文本相似度和聚类**：利用Embedding来理解文本之间的相似度，并将相似的评论聚类，以发现共同主题或意见；
- **异常检测**：通过不同用户（UserId）、用户昵称（ProfileName）、评论时间（Time）等多个字段识别出异常的评论，比如检测出不真实的（可能是机器生成的）评论或者是操纵评分的行为；

&emsp;&emsp;而这也是我们选择该数据集作为后续所有案例的基本数据的根本原因，它不仅适合于开展不同的自然语言处理（NLP）任务，也非常适合探索Embedding在不同NLP任务中的应用效果。接下来，我们就**尝试通过巧妙地将Embedding技术应用于融入到NLP场景中，来了解Embedding的应用技巧和体会其存在的实际价值**。

&emsp;&emsp;在深入了解亚马逊精选食品评论数据集之后，在实践之前，还有一项重要且关键的步骤需要完成，就是**数据集的预处理**。这是出于以下几点考虑：首先，**虽然原始数据集提供了丰富的字段，但并非所有字段都对我们即将展开的案例分析有实际价值，因此，我们将筛选出对我们分析有帮助的信息，移除不必要的数据列**。其次，鉴于整个数据集包含56万+条评论，若**直接使用全量数据，在调用 OpenAI 的 Embedding API 时会产生相对较高的费用，且运行效率较低，出于课程教学的实际需求，我们将从原始数据集中提取最新的1,000条评论**，形成一个比较高效的子集供后续使用。最后，为了确保 Embedding 技术能够发挥最佳效果，我们还会对选定的数据进行适当的预处理，以提升数据质量和分析的准确性。

> 为了便于大家跟随课程案例进行实操，我们已经将所需的数据集上传到了百度网盘，大家可以通过提供的链接下载数据集到本地环境：[网盘](https://pan.baidu.com/s/1ubLgPNa4WLFn9aYfroXn-g?pwd=6mg5) 

- **Step 1.导入第三方库**

&emsp;&emsp;首先，我们先统一导入案例中需要用到的一些第三方库，以确保代码能够顺利执行。

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import tiktoken
import time
from pprint import pprint

from openai.embeddings_utils import get_embedding

from sklearn.manifold import TSNE
from ast import literal_eval
from sklearn.decomposition import PCA

# imports
from ast import literal_eval
from sklearn.metrics import classification_report

from openai.embeddings_utils import cosine_similarity, get_embedding
from sklearn.metrics import PrecisionRecallDisplay

from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from openai.embeddings_utils import plot_multiclass_precision_recall

import matplotlib.pyplot as plt
from openai.embeddings_utils import cosine_similarity  

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns

import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

- **Step 2.读取数据集**

&emsp;&emsp;这里读取的数据集是我们从原数据集中筛选出来的子数据集，即仅包含最新的1,000条评论。

In [3]:
# 使用 1,000 条最新评论的子集
input_datapath = "00_data/01_Base/fine_food_reviews_1k.csv"   # 注意，请将此路径替换为数据集的实际本地存放路径
df = pd.read_csv(input_datapath, index_col=0)
df.head(5)

,Time,ProductId,UserId,Score,Summary,Text
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos..."
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...


&emsp;&emsp;在这个英文数据集，`Text`字段的评论具有明确的情感倾向，比如"Not pleased at all" (一点也不高兴)、"I like the fact"（我喜欢这一点）....，通过如下所示的饼图，可以直观地看到我们抽取的这1000份数据中评分的分布情况。

In [ ]:
df['Score'].value_counts().sort_index().plot(kind='pie', title='Score Distribution', figsize=(4, 4), autopct='%1.2f%%')

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231103151609001.png"></div> 

&emsp;&emsp;能够看出，在筛选得到的1000条样本中，5星级评价的占比最高，为65.10%，而2星级评价的占比最低，仅为4.90%。除此之外，大家也可以通过其他多种方法来深入分析和探索数据集的更多特性。

- **Step 3.数据预处理**

&emsp;&emsp;在预处理的过程中，首先我们需要剔除掉一些对我们后续分析没有实际意义的字段，仅保留以下5个字段：
- ProductId：产品的唯一识别码；
- UserId：用户的唯一识别码；
- Score：产品的评分星级，介于1到5之间；
- Summary：对评论内容的简短总结；
- Text：用户对产品的具体评价内容；

In [5]:
df = df[["ProductId", "UserId", "Score", "Summary", "Text"]]             # 使用主要列
df = df.dropna()
df.head(5)

,ProductId,UserId,Score,Summary,Text
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...
1,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos..."
2,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...
3,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...
4,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...


&emsp;&emsp;将`Summary`字段和`Text`字段合并成一个新的字段`combined`，也就是说我们要将评论的简短总结和评论的具体内容合并为一个组合文本。代码如下：

In [6]:
# '\n' 会影响Embedding结果，直接用空格代替
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()     # 合并字段
)
df.head(5)

,ProductId,UserId,Score,Summary,Text,combined
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...
2,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ...."
3,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...
4,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...


> 在实际应用中我们发现，当做文本Embedding时，如果将输入文本中的换行符（\n）替换为空格可以得到更好的效果，因为包含换行符时，结果可能会受到负面影响。

- **Step 4.设置 OpenAI Embedding 模型参数**

&emsp;&emsp;这里我们使用OpenAI的第二代Embedding模型`text-embedding-ada-002`来获取评论文本(组合文本`combined`)的Embedding表示，因为`text-embedding-ada-002`的最大输入长度是8191，为了避免评论文本因超出最大限制而被意外截断，我们设置`max_tokens` 为8000，超过这个数值的文本，将不再使用。参数设置如下：

In [7]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

- **Step5.计算数据集中每一行`combined`字段所占用的Tokens**

&emsp;&emsp;使用tiktoken.get_encoding方法，计算出组合文本`combined`列中每项内容占用的Tokens，如果小于我们设置的max_tokens = 8000，则将其作为一个新的列n_tokens记录下来。

In [9]:
top_n = 1000

encoding = tiktoken.get_encoding(embedding_encoding)  # embedding token 计数

# 抽取 token_len 小于 max_tokens
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)

df.head(5)

,ProductId,UserId,Score,Summary,Text,combined,n_tokens
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52
1,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,35
2,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....",267
3,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,239
4,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,86


In [10]:
len(df)

1000

&emsp;&emsp;能够发现，我们筛选出来的1000条数据子集中，组合文本`combined`列中文本内容的Tokens全部小于`text-embedding-ada-002`模型的最大输入长度，其中`n_tokens`列中明确计算出了每条评论内容在`cl100k_base`编码后的Token数量。

- **Step 6.对组合文本`combined`进行Embedding编码**

&emsp;&emsp;我们可以先按照在`2.3 text-embedding-ada-002模型的调用费用估算`中介绍的 OpenAI的 Embedding API的费用计算方式，在实际调用前进行一个具体的测算。

In [20]:
# 计算花费
print('Total tokens :%d, $ %.6f'%(df['n_tokens'].sum(), df['n_tokens'].sum() * 0.0001/1000))

Total tokens :95895, $ 0.009590


&emsp;&emsp;可以看出，该数据集的组合文本`combined`列中一共有95895个Tokens需要进行Embedding，需要花费0.009590美元。

&emsp;&emsp;在明确了此次调用的费用后，接下来我们开始对组合文本`combined`进行编码，输出单个向量Embedding，最后将结果存储到本地。

&emsp;&emsp;需要说明的是，在`2.2.1 text-embedding-ada-002模型的本地调用测试`中，我们已经介绍了如何通过OpenAI的Embedding API Python 端点（EndPoint）实现文本的Embedding。除此之外，还可以利用`openai.embeddings_utils`提供的`get_embedding`函数直接获取文本的Embedding结果。这里我们选择直接使用get_embedding函数。

&emsp;&emsp;先尝试使用一条数据进行测试，代码如下：

In [23]:
# 取 df 数据集中的第一行数据，作为测试数据
df_embedding_test = df.head(1)

df_embedding_test

,ProductId,UserId,Score,Summary,Text,combined,n_tokens
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52


In [26]:
pprint(df_embedding_test["combined"].to_list())

# ('Title: where does one  start...and stop... with a treat like this; Content: '
#  'Wanted to save some to bring to my Chicago family but my North Carolina '
#  'family ate all 4 boxes before I could pack. These are excellent...could '
#  'serve to anyone')

['Title: where does one  start...and stop... with a treat like this; Content: '
 'Wanted to save some to bring to my Chicago family but my North Carolina '
 'family ate all 4 boxes before I could pack. These are excellent...could '
 'serve to anyone']


> pprint 是Python的一个模块，它提供了格式化输出的功能，特别是当输出内容较长或者结构较复杂时，pprint能够提供更加阅读友好的格式。

In [29]:
# 使用 get_embedding 函数对 组合文本 combined 进行编码，并将结果存储在df_embedding_test的新列embedding中。
df_embedding_test["embedding"] = df_embedding_test.combined.apply(lambda x: get_embedding(x, engine=embedding_model)) 

C:\Users\Admin\AppData\Local\Temp\ipykernel_4280\3784999677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_embedding_test["embedding"] = df_embedding_test.combined.apply(lambda x: get_embedding(x, engine=embedding_model))


> 注意：执行此操作前，请确保已经正确加载 openai.api_key

In [30]:
df_embedding_test

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007060592994093895, -0.02732112631201744, 0..."


&emsp;&emsp;能够发现，通过get_embedding，我们已经成功获取到组合文本`combined`对应的Embeding表示。

&emsp;&emsp;在测试成功后，我们对全部的1000条数据集中的组合文本`combined`进行Embedding编码，并将最终结果保存为本地的.csv文件。

In [31]:
# get_embedding 获取embedding编码
# 1000 条评论将花费10mins左右
# df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))   
# df.to_csv("ttachment/Amazon_Fine_Food_Reviews/fine_food_reviews_with_embeddings_1k.csv")

> 编码过程需要大约10分钟的时间，我们已经预先完成了编码工作，并在网盘中提供了相应的csv文件供下载，大家可以自行执行这一过程或直接使用我们提供的转换后文件。

- **Step 7. 查看Embedding编码数据集**

In [32]:
# 注：此处需要将读取文件路径替换为本地实际的文件存储路径
df = pd.read_csv("00_data/01_Base/fine_food_reviews_with_embeddings_1k.csv", index_col="Unnamed: 0")
df.head()

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007018072064965963, -0.02731654793024063, 0..."
297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.003140551969408989, -0.009995664469897747,..."
296,B008JKTTUA,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,..."
295,B000LKTTTW,A14MQ40CCU8B13,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...,111,"[-0.0013932279543951154, -0.011112828738987446..."
294,B001D09KAM,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,..."


&emsp;&emsp;至此，我们就完成了Embedding的数据集准备工作，接下来，我们将使用这个数据集来进行后续案例的实践。

### 四、基于Embedding的零样本分类实现流程

&emsp;&emsp;**零样本学习 (Zero-Shot Learning, ZSL)** 是一种机器学习范式，在这种范式中，模型被训练来处理它在训练阶段从未见过的类别。传统的机器学习和深度学习方法通常需要每个类别都有大量的标注数据来学习，但在许多实际应用中，对某些类别的数据进行收集和标注可能是困难的或代价高昂的。ZSL的目的是利用已有的知识来识别、分类或处理这些没有标注样本的新类别。而**零样本分类**，特指在分类任务中应用**零样本学习**的概念，它希望做到的是：利用已知类别的信息来正确分类未知类别的实例。

&emsp;&emsp;在这个任务中，Embedding也能够发挥出比较关键的作用，因为Embedding本身就具备语义信息，比如像“犬”和“狗”这样语义上相近的词语会被映射到接近的点，如果通过Embedding将与类别相关的辅助信息（例如类别的描述或者属性）转化为一个连续的向量空间，这样，即使模型在训练时没有见过某个类别的样本，它也可以通过这个语义空间中的位置来识别或分类这个类别。

> 对于我们的评论数据，与类别相关的辅助信息可能包括评论的情感、主题或其他与评论相关的描述性信息。例如，对于一个产品的评论，辅助信息可能是产品的类别、功能或其他属性。

&emsp;&emsp;为了实现这一点，我们可以首先对每个标签的描述（如“正面”和“负面”）进行Embedding转换，然后我们计算每个评论与这些分类描述之间的余弦相似度，评论与哪个分类标签的描述更接近，那么该评论就更可能属于该分类。同时为了使结果更具解释性，我们还可以设计一个预测分数，该分数是评论与“正面”标签的余弦相似度与其与“负面”标签的余弦相似度之差，如果这个差值大于0，标识为"积极"，如果小于0，标识为"消极"。

&emsp;&emsp;例如各个评论与标签的余弦值结果如下：
|评论|星级（$y$ ）|积极|消极|余弦分类（$\hat y$ ）|
|:----:|:----:|:----:|:----:|:----:|
|A|5|0.36|0.75|消极|
|B|2|0.85|0.15|积极|
|C|4|0.62|0.38|积极|
|D|1|0.29|0.71|消极|

> 需要说明的是，我们将4星和5星的评价定义为正面情绪，把1星和2星的评价定义为负面情绪，3星的评价被视为中立，在这个例子中不会使用它们。

- **Step 1.标记情感标签**

&emsp;&emsp;按照前面提到的思路，我们先过滤掉评分为3的评论，1星和2星的评价标记为“negative”，而4星和5星的评价标记为“positive”。

In [48]:
# 注：此处需要将读取文件路径替换为您本地实际的文件存储路径
datafile_path = "00_data/01_Base/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)

# 选取1245评分并归为积极、消极评分
df = df[df.Score != 3]
df["sentiment"] = df.Score.replace({1: "negative", 2: "negative", 4: "positive", 5: "positive"})

In [49]:
df.head(5)

,Unnamed: 0,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding,sentiment
0,0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007018072064965963, -0.02731654793024063, 0...",positive
1,297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.003140551969408989, -0.009995664469897747,...",positive
2,296,B008JKTTUA,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,...",negative
3,295,B000LKTTTW,A14MQ40CCU8B13,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...,111,"[-0.0013932279543951154, -0.011112828738987446...",positive
4,294,B001D09KAM,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,...",negative


- **Step 2.样本分类预测及可视化**

&emsp;&emsp;在这个过程中，我们首先需要再次调用`get_embedding`函数，先获取到标签（negative和positive）的Embedding，然后计算给定评论Embedding与正面标签Embedding的余弦相似度与其与负面标签Embedding的余弦相似度之差，这个差值如果大于0，就归为positive类，如果小于0，就归为negative类，并绘制精确度-召回率曲线。具体代码如下：

In [ ]:
# 设置Embedding模型名称
EMBEDDING_MODEL = "text-embedding-ada-002"

# 定义零样本分类的评估函数
def evaluate_embeddings_approach(
    labels = ['negative', 'positive'], 
    model = EMBEDDING_MODEL,
):
    # 获取标签的Embedding
    label_embeddings = [get_embedding(label, engine=model) for label in labels]

    # 定义标签评分函数
    def label_score(review_embedding, label_embeddings):
        # 计算给定评论Embedding与正面标签Embedding的余弦相似度与其与负面标签Embedding的余弦相似度之差
        return cosine_similarity(review_embedding, label_embeddings[1]) - cosine_similarity(review_embedding, label_embeddings[0])
        
    # 计算每个评论的评分
    probas = df["embedding"].apply(lambda x: label_score(x, label_embeddings))
    # 基于评分做出最终的预测情感
    preds = probas.apply(lambda x: 'positive' if x>0 else 'negative')

    # 打印分类报告
    report = classification_report(df.sentiment, preds)
    print(report)

    # 绘制精确度-召回率曲线
    display = PrecisionRecallDisplay.from_predictions(df.sentiment, probas, pos_label='positive')
    _ = display.ax_.set_title("2-class Precision-Recall curve")

evaluate_embeddings_approach(labels=['negative', 'positive'], model=EMBEDDING_MODEL)

&emsp;&emsp;首先来看下分类报告：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231103172908846.png"></div> 

- 对于**负面**评论：
    - **精确度(Precision)**: 0.61，意味着模型预测为负面的评论中，有61%确实是负面评论;
    - **召回率(Recall)**: 0.88，说明能够检测到88%的实际负面评论;
    - **F1得分**: 0.72，是精确度和召回率的调和平均值，为模型性能提供了一个整体评价;
    - **样本(Support)**: 136，意味着测试数据中一共有136条负面评论;
<br><br>
- 对于**正面**评论：
    - **精确度**: 0.98，意味着模型预测为正面的评论中，有98%确实是正面评论;
    - **召回率**: 0.90，说明能够检测到90%的实际正面评论;
    - **F1得分**: 0.94;
    - **样本**: 789，意味着测试数据中有789条正面评论;
<br><br>
- **总体评价**：
    - **准确率(Accuracy)**: 0.90，意味着模型对90%的评论做出了正确的预测;
    - **宏平均(Macro avg)** 和 **加权平均(Weighted avg)** 分别对各个标签的评价指标进行了平均，用于评估模型在整体上的性能;

&emsp;&emsp;再看下精确度-召回率曲线：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231103172918291.png"></div> 

&emsp;&emsp;从图中可以看到，随着召回率的提高，精确度在某些点上有所下降。这是因为为了捕获更多的正样本，可能会误判一些负样本。理想的曲线应该尽可能地靠近图的右上角，意味即使召回率很高，精确度也保持在高水平，在本图中，大部分时间的精确度都保持在较高的水平，尤其是在召回率较高时，这说明模型的性能相当不错。

&emsp;&emsp;总体而言，当前的分类器性能已经相当出色，特别是考虑到我们只利用了简单的相似性嵌入，并用最基本的标签名称进行分类。当然，我们还可以进一步提高分类器的准确性，因为“积极”和“消极”描述过于简洁，我们可以尝试优化为更丰富的：
- negative（消极） --> 'An Amazon review with a negative sentiment.'（带有消极情绪的亚马逊评论）
- positive（积极） --> 'An Amazon review with a positive sentiment.'（带有积极情绪的亚马逊评论）

In [ ]:
evaluate_embeddings_approach(labels=['An Amazon review with a negative sentiment.', 'An Amazon review with a positive sentiment.'])

&emsp;&emsp;其对应的分类结果如下所示：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231103174740807.png"></div> 

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231103174749012.png"></div> 

&emsp;&emsp;从结果上看，经过简单优化后的分类器在多个指标上都有所提升，尤其是负面评论的精确度和整体的F1-得分。虽然负面评论的召回率有所下降，但考虑到精确度的显著提高，整体性能仍然更为出色。所以**Embedding在进行零样本分类任务时，对模型效果的提示是非常显著的，尤其是在分类标签更丰富、更具描述性的情况下。**

###  五、将Embedding作为文本特征编码器并进行有监督学习

&emsp;&emsp;如果学习过我们的机器学习课程，或有机器学习基础的同学应该比较清楚，在机器学习的算法建模中，通常会经历三个关键阶段：
- Stage 1.业务背景解读与数据探索
- Stage 2.数据预处理与特征工程
- Stage 3.算法建模与模型调优<br>

&emsp;&emsp;特别是在特征工程阶段，我们专注于优化和转换数据特征，使得数据的内在规律更易于被模型捕捉和学习，这一步骤对于提升模型的训练效果至关重要。在处理特征时，除了必须对离散变量和连续变量进行区分和处理外，另一个重要的任务是有效地处理文本型变量。

&emsp;&emsp;需要说明的是，为了确保模型能在一组独立的数据上进行公平的评估，我们会将数据集分为训练集和测试集。

#### 5.1 回归任务建模流程

&emsp;&emsp;我们首先来看回归任务。回归意味着预测一个数字，而不是其中一个类别，我们的目标是预测产品评论的评分星级，这个评分星级是一个介于1到5的连续数字，它表征了用户对产品的满意程度。其中，1分代表用户的强烈不满（负面评价），而5分则表示用户的高度认可（正面评价）。

&emsp;&emsp;我们使用在`预处理统一的数据集并获取评论文本的Embedding表示`中生成的文本Embedding向量，作为特征输入到一个随机森林回归器中，以预测评论的评分星级。具体步骤如下：

- **Step 1.划分训练集和测试集**

&emsp;&emsp;为确保模型评估的客观性和有效性，我们遵循机器学习建模的标准实践流程，将数据集细分为训练集和测试集。

In [2]:
# 注：此处需要将读取文件路径替换为您本地实际的文件存储路径
datafile_path = "00_data/01_Base/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)

# 将数据拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(list(df.embedding.values), df.Score, test_size=0.2, random_state=42)

In [5]:
# 打印X_train的前几行
for i in range(min(len(X_train), 5)):  # 假设打印前5行
    print(X_train[i])

[-0.00517403 -0.01152934 -0.00617981 ... -0.0272843  -0.03099691
 -0.02211365]
[ 0.00681022 -0.00415957  0.00091721 ... -0.00101986  0.0032432
 -0.01670506]
[ 0.01110131  0.00775589  0.01700015 ... -0.01077359 -0.01710939
 -0.02900267]
[-0.01242283 -0.01769373  0.00818883 ... -0.00598875 -0.00985053
 -0.02468614]
[ 0.00906392 -0.03056136 -0.00721588 ... -0.0165606  -0.01499335
 -0.0107226 ]


- **Step 2.构建随机森林模型并预测**

&emsp;&emsp;选择随机森林回归器作为预测模型，是因为它能够处理较大的特征空间并提供重要的非线性建模能力。通过计算均方误差（MSE）和平均绝对误差（MAE），我们可以量化模型预测的准确性和可靠性，这些指标反映了预测评分与实际评分之间的偏差大小。

In [6]:
rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X_train, y_train)
preds = rfr.predict(X_test)

mse = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(f"ada-002 embedding 表现: mse={mse:.2f}, mae={mae:.2f}")

ada-002 embedding 表现: mse=0.63, mae=0.53


> MSE 和 MAE 的值越低，表示模型对数据的拟合越好。

&emsp;&emsp;通过初步的建模尝试，我们利用随机森林回归器，在未经参数调优和模型复杂化的情况下，得到了一个均方误差（MSE）为0.63和平均绝对误差（MAE）为0.53的结果。这个MAE意味着我们的模型在预测用户评分时，平均每次预测大约有半个评分星级的误差，对于一个五分制的评分系统，这样的误差水平意味着这个模型能够较为准确地预测一半的用户评分星级，而另一半的预测结果会有大约一个星级的偏差。

&emsp;&emsp;但是，这样的一个效果到底是好是坏呢？我们需要一个评估标准，为此，我们可以尝试构建一个Baseline Model。

> Baseline model，即基准模型，是指一个简单的模型，它不依赖于任何复杂的机器学习算法来做出预测。

- **Step 3.构建Baseline Model**

&emsp;&emsp;这里我们采用一种比较简单的方法来构建Baseline Model：预测所有值为目标变量的平均值。我们可以把它看作是一个“无信息”预测，也就是说，如果不知道关于输入数据的任何信息，最佳猜测就是目标值的历史平均值。计算方法也比较简单：

In [7]:
bmse = mean_squared_error(y_test, np.repeat(y_test.mean(), len(y_test)))
bmae = mean_absolute_error(y_test, np.repeat(y_test.mean(), len(y_test)))
print(
    f"平均预测效果: mse={bmse:.2f}, mae={bmae:.2f}"
)

平均预测效果: mse=1.73, mae=1.03


&emsp;&emsp;能够发现，Baseline Model的MSE 1.73和MAE 1.03，预测误差几乎是随机森林模型的两倍，而其表现出的实际意义也是预测结果平均偏差超过了一个星级。尽管我们构建的Baseline Model的模型思想比较简单，它仅仅是预测每个评论的评分星级为所有评论评分的平均值，但这样的模型设计在评估和比较中仍具有一定的价值，也在一定程度上帮助我们量化和理解了Embedding应用在ML中的效果提升。


&emsp;&emsp;**总的来说，当Embedding作为ML回归任务的文本特征编码器，预测精度是相当不错的，尤其是在考虑到评分是主观且可能受到多种因素影响的条件下**。

#### 5.2 分类任务建模流程

&emsp;&emsp;在上一小节中，我们将评分星级当作一个连续变量来建模，期望预测出一个尽可能接近实际值的分数。如果我们改变策略，把每一种可能的评分星级当作一个单独的类别来处理，那么原本的回归问题就演变为了分类问题。在这种模式下，模型的任务转变为识别和判定每个评论所对应的最可能的星级类别。具体来说，我们需要将1至5颗星的评分看作是5个不同的类别，并让模型预测每条评论属于这5个类别中的哪一个。建模过程如下：

&emsp;&emsp;我们还是使用在上一小节已经切分好的训练集和测试集，不同的是，这次我们将使用随机森林的分类器。

In [ ]:
# train random forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

&emsp;&emsp;看下输出结果：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231106163004103.png"></div> 

&emsp;&emsp;能够看到，随机森林分类模型已经能够较好的区分各个类别，从分类报告上来看：

1. **评价星级 1:**
    - 精确度（Precision）: 1.00 表示模型预测为评价星级为 1 的所有样本中，100% 都被正确分类。
    - 召回率（Recall）: 0.25 表示所有实际上属于评价星级为 1 的样本中，只有 25% 被模型正确找到并分类。
    - F1得分: 0.40 是精确度和召回率的调和平均，考虑到二者的平衡，得分较低，说明模型在这个评价星级上的表现不佳。
    - 样本数（Support）: 20 表示实际上有 20 个样本属于评价星级 1。
<br>
<br>
2. **评价星级 2、3、4:**
    - 这几个类别的情况与评价星级 1 类似，精确度较高，但召回率和F1得分较低，说明模型在这几个类别上的预测能力有待提升。
<br>
<br>
- **评价星级 5:**
    - 精确度稍低（0.74），但召回率非常高（1.00），说明模型倾向于将样本分类为评价星级 5，这可能导致其他类别的召回率降低。
    - F1得分较高（0.85），说明在评价星级 5 上的预测表现较好。
    - 样本数为 134，说明评价星级 5 的样本量较大。
<br>

&emsp;&emsp;整体来看，模型在1星至4星的评分类别上表现欠佳，尤其是在召回率上，所以对于较低星级的评论的预测正确性有待提高。然而，在5星评级上，模型的表现相对最佳，这可能是因为5星评价在数据集中出现频率较高。尽管存在类别不平衡的问题，但模型整体的准确率达到了76%，这显示出模型具备相对较好的分类能力。

&emsp;&emsp;除此之外，我们还可以使用OpenAI Embedding API 提供的plot_multiclass_precision_recall函数绘制多类别的精确度-召回率曲线图，更加直观的看到模型在各个星级类别上的性能。

In [ ]:
plot_multiclass_precision_recall(probas, y_test, [1, 2, 3, 4, 5], clf)

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231106170347050.png"></div> 

&emsp;&emsp;上图展示了不同星级评分的精确度-召回率曲线，每个曲线下方的阴影部分代表了平均精确度-召回率曲线（AUPRC），数值越高，表明模型在对应类别上的性能越好。

&emsp;&emsp;可以很明显的看到，模型在所有类别上的平均精确度（Average precision score over all classes）为88%，这是一个相当高的得分。同时，其他的结论也与分类报告中的分析基本保持一直，如星级5的表现依然是最好的，具有最高的AUPRC值（0.97）。

&emsp;&emsp;**综上，Embedding技术不仅限于自由文本的特征编码，在处理分类特征时同样显示出其独特的优势。当我们处理的分类变量本身具有丰富的语义信息且种类繁多时（如不同的职务名称），Embedding能够有效地捕捉这些变量之间的潜在关系和层次结构。此外，Embedding表示通常信息量大，与使用简化维度的方法如SVD或PCA相比，它能在不损失关键信息的前提下维持甚至提升模型性能。**

###  六、借助Embedding进行聚类分析

&emsp;&emsp;在数字化时代，用户生成的评论是企业获取客户反馈的有效渠道之一。比如某东、某宝购买商品后的评价，某团的用餐评价，这些评论的文本数据是理解消费者行为和偏好的关键，然而，面对海量且多样的用户评论，如果手动进行分析，不仅耗时费力，而且效率低下。在这样的背景下，聚类分析这种无监督学习技术显得尤为重要，聚类可以将具有相似语义内容的评论归并到相同的类别中，并不需要依赖预先定义的分类。通过对分类后的数据进行人工分析，就能够从繁杂的评论数据中识别出对产品或服务的普遍好评与差评，抽象出数据的核心价值，进而揭示消费者评价背后的潜在模式。

&emsp;&emsp;**而Embedding 为聚类算法提供了一种量化文本相似度的方式，使其可以基于向量间的距离或相似度将文本归纳到不同的类别中。**

&emsp;&emsp;以我们所使用的亚马逊精选美食评论数据集来说，在经过`预处理统一的数据集并获取评论文本的Embedding表示`中的一系列处理后，我们现在已经有了评论文本的Embedding表示，那么我们就可以直接使用聚类算法，发现用户评论中的隐含主题或趋势，自动地为用户画像划分不同的类别。

&emsp;&emsp;在这个案例中，我们将使用K-Means聚类算法，对经过Embedding向量化的评论文本进行聚类，验证其聚类结果的有效性后，利用OpenAI的`text-davinci-003`模型对从每个类别中随机抽取出的评论样本进行主题分析，以此来检验Embedding在提高文本数据聚类质量方面的作用。详细步骤如下：

- **Step 1.数据加载及预处理**

&emsp;&emsp;本案例使用的数据集还是经过Embedding后的1000条数据集，但需要注意，在进行K-Means建模之前，我们需要将字符串格式的Embedding向量合并成一个连续的NumPy数组，即二维矩阵，使其能够被K-means等算法有效处理。代码如下：

> 在使用K-means聚类算法时，数据通常需要以数值矩阵的形式提供。这是因为K-means算法在操作时会计算数据点（在本案例中为Embedding）之间的距离。

In [16]:
# 注：此处需要将读取文件路径替换为您本地实际的文件存储路径
datafile_path = "00_data/01_Base/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path) # 读取所有的 embedding 是长得像列表的字符串
# print(type(df.embedding[0]))  # 字符串类型
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)  # 将字符串转换为 np
matrix = np.vstack(df.embedding.values) # 将 pd.DataFrame 转为 np.ndarray

In [17]:
df["embedding"]

0      [0.007018072064965963, -0.02731654793024063, 0...
1      [-0.003140551969408989, -0.009995664469897747,...
2      [-0.01757248118519783, -8.266511576948687e-05,...
3      [-0.0013932279543951154, -0.011112828738987446...
4      [-0.01757248118519783, -8.266511576948687e-05,...
                             ...                        
995    [0.00011091353371739388, -0.00466986745595932,...
996    [-0.020869314670562744, -0.013138455338776112,...
997    [-0.009749102406203747, -0.0068712360225617886...
998    [-0.00521062919870019, 0.0009606690146028996, ...
999    [-0.006057822611182928, -0.015015840530395508,...
Name: embedding, Length: 1000, dtype: object

In [18]:
matrix

array([[ 7.01807206e-03, -2.73165479e-02,  1.05734831e-02, ...,
        -7.01120170e-03, -2.18614824e-02, -3.75671238e-02],
       [-3.14055197e-03, -9.99566447e-03, -3.48033849e-03, ...,
        -9.74494778e-03, -2.39829952e-03, -9.20392852e-03],
       [-1.75724812e-02, -8.26651158e-05, -1.15222773e-02, ...,
        -1.39020244e-02, -3.90170924e-02, -2.35151257e-02],
       ...,
       [-9.74910241e-03, -6.87123602e-03, -5.70622832e-03, ...,
        -3.00459806e-02, -8.14515445e-03, -1.95114054e-02],
       [-5.21062920e-03,  9.60669015e-04,  2.82862745e-02, ...,
        -5.38039953e-03, -1.33138765e-02, -2.71892995e-02],
       [-6.05782261e-03, -1.50158405e-02, -2.07575737e-03, ...,
        -2.90671214e-02, -1.41164539e-02, -2.28756946e-02]])

In [19]:
matrix.shape

(1000, 1536)

&emsp;&emsp;能够发现，我们已经将原始的DataFrame中的Embedding列转换成了一个NumPy数组，并将这些数组堆叠成了一个1000x1536的矩阵，其中每一行代表一个评论的Embedding向量，可以直接用于后续的聚类算法，其中：
- `1000`代表矩阵有1000行，这与数据集中的评论数相对应，因为我们使用的是包含最新1000条评论的子数据集；
- `1536`代表矩阵有1536列，这表示每个Embedding向量的维度是1536，因为OpenAI Embedding API 返回的Embedding维度就是1536；

- **Step 2.K-Means建模**

&emsp;&emsp;我们使用K-Means聚类算法来对Step 1 中获取到的Embedding矩阵进行聚类分析。具体来说，设置`n_clusters`参数来训练K-Means模型，训练完成后，对每个评论赋予一个标签，指示该评论属于哪一个类别。这里我们设置为聚类的数量`n_clusters`为4，表示我们期望算法将数据分成4个不同的类别，代码如下：

In [24]:
n_clusters = 4 # 分四个簇

kmeans = KMeans(n_clusters=n_clusters, init="k-means++", random_state=42) # 实例化 KMeans
kmeans.fit(matrix) # 拟合
labels = kmeans.labels_ 
df["Cluster"] = labels # 对标签赋值

D:\software\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\software\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


- **Step 3.聚类结果分析**

In [27]:
df.head(5)

,Unnamed: 0,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding,Cluster
0,0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007018072064965963, -0.02731654793024063, 0...",3
1,297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.003140551969408989, -0.009995664469897747,...",2
2,296,B008JKTTUA,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,...",3
3,295,B000LKTTTW,A14MQ40CCU8B13,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...,111,"[-0.0013932279543951154, -0.011112828738987446...",0
4,294,B001D09KAM,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,...",3


In [28]:
df['Cluster'].unique()

array([3, 2, 0, 1])

&emsp;&emsp;能够发现，经过K-Means后，每个评论都被划分到了不同的类别中，因为我们训练时设置的`n_clusters`为4，所以新增的`Cluster`这一列的取值是0~3。为了直观展现每个类别含有的评论数量，我们可以采用可视化的手段来进行展示。代码如下：

In [31]:
df['Cluster'].value_counts()

Cluster
0    359
2    269
3    236
1    136
Name: count, dtype: int64

In [ ]:
df.groupby(["Cluster"])["Text"].count().plot(kind='pie', title='Score Distribution', figsize=(4, 4), autopct='%1.2f%%')

&emsp;&emsp;可视化结果如下：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231107144707303.png"></div> 

&emsp;&emsp;如上图所示，这张饼状图直观地展示了K-Means聚类算法将评论分为四个类别后的结果。从分布数量上来看，这是一个不均匀的分布，其中类别0包含了最多的评论（约36%），而类别1则包含了最少的评论（约14%）。类别中评论数量的差异可能暗示了聚类的质量。一个较好的聚类结果通常希望在各个类别之间有比较均衡的数据点分配，如果某个类别中的样本数量特别大，可能意味着聚类过程将过多不同的点归为同一类，但也可能确实反映了数据中的一个主要趋势。所以这样的效果是好还是坏，我们需要进行更进一步的探索来验证。

&emsp;&emsp;为了评估聚类结果的有效性，我们可以先尝试计算每个类别内的评论平均得分，通过这个得分，可以大致判断不同类别可能对应的评论质量或用户满意度，帮助理解聚类是否按照某种有意义的模式或者趋势组织了数据点，比如看看是否高评分和低评分的评论被有效地区分开来。代码如下：

In [30]:
# 计算聚类后每种标签的平均得分
df.groupby("Cluster").Score.mean().sort_values() 

Cluster
3    4.025424
1    4.191176
2    4.215613
0    4.353760
Name: Score, dtype: float64

&emsp;&emsp;每个类别的平均评分提供了一种初步的、量化的方式来理解每个簇中评论的性质。虽然没有具体的文本数据来定义每个类别代表的具体内容，但平均得分的高低可以作为判别不同类别用户评论水平的一个指标。从输出结果上看：

- **类别0**的平均得分最高（约4.35），可能表明该类别包含了相对正面的评论；
- **类别1和类别2**的平均得分相对较高（约4.19和4.21），可能表明这些类别中的评论也趋于正面，但可能包含一些中性的意见；
- **类别3**的平均得分最低（约4.03），可能表明它包含更多的负面评论；

&emsp;&emsp;需要说明的是：上述我们均提到"可能表明"一词，这说明上述结论仅为我们人工的初步判断，实际上，这些得分并不能完全确切地告诉我们每个类别的具体特征或用户评论的具体内容。要想深入了解每个类别代表的内容，我们需要更进一步的查看每个类别中的代表性评论，以获取更具体的信息。

&emsp;&emsp;同样，我们还是可以先对不同聚类标签中评论的评分分布尝试进行可视化分析。具体来说，按照聚类结果的标签和评论的评分进行聚合，计算出每个聚类标签和评分组合下的评论数量，可以直观的看出在不同的聚类标签中，各个评分段的评论是如何分布的，通过这样的设计，我们可能会发现一些有趣的结果，例如某个聚类标签可能会有异常多的高分或低分评论。代码如下：

In [ ]:
# 先对数据进行聚合
grouped_data = df.groupby(["Cluster", 'Score'])["Text"].count().reset_index()

# 使用 seaborn 绘制柱状图
plt.figure(figsize=(6,4))
sns.barplot(x='Cluster', y='Text', hue='Score', data=grouped_data)

plt.title('Score distribution')
plt.xlabel('labels')
plt.ylabel('counts')
plt.show()

&emsp;&emsp;可视化结果如下图所示：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231107151532832.png"></div> 

In [38]:
# 计算每个Score的数量
df['Score'].value_counts().sort_index()

Score
1     87
2     49
3     75
4    138
5    651
Name: count, dtype: int64

> Score字段代表评价星级，即满意度 Star 1 ~ Star 5 

&emsp;&emsp;结合图表和Score数量的统计结果能够发现，在我们使用的数据集中，评论星级5的数量最多，且占有非常大的比重，这直接导致了在聚类后的各个类别中，评论星级5的占比也都是最大的。这就很好的解释了为什么我们在计算聚类后每种标签的平均得分都是很接近的。因为评论星级5的普遍性降低了其他星级评价数量差异带来的影响。更具体地说，由于数据集中评论星级5的数量非常大，使得即使是分布较少的一星和二星评价，也难以在整体聚类的平均评分中形成明显差异。


&emsp;&emsp;综上所述，我们能够得出的初步结论是，尽管聚类后的每个类别平均得分较为接近，但这种接近程度在很大程度上受到五评论星级5数量占主导地位这一事实的影响。这一点在我们的数据集中特别明显，因为总体上的平均评分为$1*87+2*49+3*75+4*138+5*651=4.217$ ，这与我们通过聚类得出的各个标签的平均评分相吻合。为了深入了解聚类标签背后的含义，我们接下来分析评论的文本内容，准确地了解不同聚类所代表的用户偏好和意见。

&emsp;&emsp;在分析文本前，对Embedding做可视化是一个关键的前置步骤。好的聚类结果通常会在可视化中展现为明显分离的簇，如果可视化结果显示聚类间有重叠或混乱，那么可能需要调整聚类方法或参数。如果连聚类的效果都不好，那么做再多的文本分析，也是毫无意义的。这个可视化过程基于与`4.3 案例一：Embedding的 2D 数据可视化`中案例的逻辑一致，这里我们使用t-SNE先将1536的维度降为2维，每个点的x和y坐标是由t-SNE算法计算出的，代表原始高维空间中的文本数据，最后，我们根据不同的聚类标签（Cluster）为点着色，以区分数据点所属的聚类。代码如下：

In [ ]:
tsne = TSNE(n_components=2, perplexity=15, random_state=42, init="random", learning_rate=200)
vis_dims2 = tsne.fit_transform(matrix)

x = [x for x, y in vis_dims2]
y = [y for x, y in vis_dims2]
legends_ = []

for category, color in enumerate(["purple", "green", "red", "blue"]):
    legends_.append(category)
    xs = np.array(x)[df.Cluster == category]
    ys = np.array(y)[df.Cluster == category]
    plt.scatter(xs, ys, color=color, alpha=0.3)
    legends_.append(category)
    avg_x = xs.mean()
    avg_y = ys.mean()

    plt.scatter(avg_x, avg_y, marker="x", linewidths=3 ,color=color, s=100)
plt.legend(legends_)
plt.title("Clusters identified visualized in language 2d using t-SNE")

&emsp;&emsp;可视化结果如下：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/image-20231107160424840.png"></div> 

&emsp;&emsp;上图显示了四个不同颜色代表的类别在二维空间中的分布，其中每个类别的数据点被标记为相同颜色。能够看出四个类别基本上是分离的，每个类别的数据点聚集在一起，形成了独立的簇。这意味着在多维空间中，各个类别内的数据点彼此相似度较高，而与其他类别的数据点相似度较低。尽管有些类别边缘的点显示出轻微的重叠，但总体上可以认为聚类的分离度是较好的。

&emsp;&emsp;现在，我们就可以根据这四个较为清晰的类别进行文本内容的分析。通过研究每个类别中的实际文本来了解：
- 每个类别的主要主题或话题是什么？
- 用户在每个类别中表达了哪些具体的情感或观点？
- 不同类别之间有哪些明显的差异？
- .........

&emsp;&emsp;我们采取的策略是：首先，从每个通过聚类算法定义的用户评论类别中随机抽取相同数量的样本（这里我们使用的是5条样本），然后，借助OpenAI的`text-davinci-003`分析抽取出来的每个样本类别中的评论的主题和共性，从而确定每个类别到底属于哪一种评论类型。代码如下：

> 注意：这里我们将`temperature`参数设置为0，以减小模型输出的不稳定性。

In [121]:
# 每个聚类标签里面抽取5条
rev_per_cluster = 5

for i in range(n_clusters):
    print(f"Cluster {i} Theme:", end=" ")

    reviews = "\n".join(
        df[df.Cluster == i]
        .combined.str.replace("Title: ", "")
        .str.replace("\n\nContent: ", ":  ")
        .sample(rev_per_cluster, random_state=42)
        .values
    )
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f'以下客户评论有什么共同点?\n\n客户观点:\n"""\n{reviews}\n"""\n\n主题:',
        temperature=0,
        max_tokens=64,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    print(response["choices"][0]["text"].replace("\n", ""))

    sample_cluster_rows = df[df.Cluster == i].sample(rev_per_cluster, random_state=42)
    for j in range(rev_per_cluster):
        print(sample_cluster_rows.Score.values[j], end=", ")
        print(sample_cluster_rows.Summary.values[j], end=":   ")
        print(sample_cluster_rows.Text.str[:70].values[j])
    print("-" * 100)

Cluster 0 Theme:  客户对产品的评价，以及对价格的抱怨。
1, Not what it used to be:   Hormel's Chili used to be an awesome pro0duct. When I was a kid, this 
5, Great Basil Taste - Organic- Gluten Free:   About time, this is the best organic gluten free basil pasta sauce I h
5, This product is great however the price of $9.32 must be a misprint.  It is available for around $3.50 on other websites.:   This product is very good on all meat products, vegetables and soups. 
5, Mrs. Dash Tomato Basil Garlic:   I saw this advertised in a magazine and was unable to find it locally.
5, Very tasty and good for you!:   I have always loved this product and was glad to subscribe so that I c
----------------------------------------------------------------------------------------------------
Cluster 1 Theme:  客户对宠物食品的评价。
2, Messy and apparently undelicious:   My cat is not a huge fan. Sure, she'll lap up the gravy, but leaves th
4, The cats like it:   My 7 cats like this food but it is a little yucky for the human. Piec

&emsp;&emsp;能够发现：
- 类别0的用户属于评价产品和抱怨价格；
- 类别1的用户属于对宠物食品评价；
- 类别2的用户属于对咖啡食品评价；
- 类别3的用户属于对口味、观感上的评价；

&emsp;&emsp;虽然每个类别中我们仅抽取了5条样本来进行测试，但从结果上看，**当把Embedding应用到聚类算法中，能够自动发现评论中的潜在主题和趋势的，将用户评论作为准确的用户画像，不仅能提高分析效率，也能极大地提升分析结果的实用性，这对数据分析工作来说，意味着通过机器学习方法，特别是无监督的聚类算法，借助Embedding技术，我们能够在短时间内从大量数据中提炼出有价值的信息，具有非常强的实际应用价值。**

###  七、借助Embedding实现文本搜索

&emsp;&emsp;文本搜索这个概念比较容易理解，实际上它就是在一个或多个文本中寻找包含特定字符串的过程，这个概念可以应用在各个层面上，从简单的文档内关键词查找，到互联网搜索引擎中复杂的信息检索，其关键目标是快速有效地找到含有搜索查询中提及的关键词或短语的文本资料。

&emsp;&emsp;但是在自然语言处理（NLP）中，文本搜索通常涉及更高级的技术，它被用以理解和处理自然语言数据。不仅仅是关键词的匹配，而是包括了对语言的深层次理解，比如语义理解、意图识别等，关键在于如何精确捕捉和处理文本的语义信息，而**Embedding 作为一种高效捕获语义信息的技术手段，借助Embedding 实现搜索查询，它可以通过语义化的方式高效且成本低廉地在大量文本中搜寻最为相关的评论。**

&emsp;&emsp;具体来说，我们将依据查询Embedding向量与评论文档Embedding向量之间的余弦相似度，最终返回与查询最为匹配的前n条结果。实现过程如下：

- **Step 1.读取数据集**

&emsp;&emsp;本案例使用的数据集还是经过Embedding后的1000条数据集，并不需要做任何的预处理。

In [40]:
# 注：此处需要将读取文件路径替换为您本地实际的文件存储路径
datafile_path = "00_data/01_Base/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)

In [41]:
df.head(5)

,Unnamed: 0,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007018072064965963, -0.02731654793024063, 0..."
1,297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.003140551969408989, -0.009995664469897747,..."
2,296,B008JKTTUA,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,..."
3,295,B000LKTTTW,A14MQ40CCU8B13,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...,111,"[-0.0013932279543951154, -0.011112828738987446..."
4,294,B001D09KAM,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248118519783, -8.266511576948687e-05,..."


- **Step 2.定义搜索函数**

&emsp;&emsp;我们需要定义一个搜索函数，来实现在评论数据集中寻找与给定搜索描述最为相似的评论的功能。具体逻辑分为以下三个核心过程：
1. 借助OpenAI的Embedding模型，将传入的搜索描述转化为Embedding向量；
2. 计算搜索描述的Embedding向量与每个评论的Embedidng向量之间的余弦相似度；
3. 根据余弦相似度对评论进行排序，选择相似度最高的前 `n` 个评论。

In [43]:
def search_reviews(df, product_description, n=3, pprint=True):
    """
        根据余弦相似度结果搜索评论最匹配的前 3 个评论
    Args
        :param                    df -> 评论数据集
        :param   product_description -> 匹配内容
        :param                     n -> 返回相似度最高的数量
    """
    product_embedding = get_embedding(
        product_description,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Title: ", "")
        .str.replace("; Content:", ": ")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results

- **Step 3.验证匹配结果**

&emsp;&emsp;search_reviews 函数接收四个参数，如下：

| 参数            | 描述                                             | 类型     | 默认值 |
|:----------------------|:---------------------------------------------------------|:----------|:---------|
| `df`                 | 数据集的DataFrame                                   | DataFrame| N/A     |
| `product_description`| 搜索描述，用于匹配评论                            | str      | N/A     |
| `n`                  | 指定返回的最相似评论数量                                | int      | 3       |
| `pprint`             | 一个布尔值，指定是否打印结果                            | bool     | True    |

&emsp;&emsp;接下来，我们将调用search_reviews函数，输入不同的搜索描述，返回最相关的前3条评论。

In [46]:
results = search_reviews(df, "delicious beans", n=3)

Good Buy:  I liked the beans. They were vacuum sealed, plump and moist. Would recommend them for any use. I personally split and stuck them in some vodka to make vanilla extract. Yum!

Jamaican Blue beans:  Excellent coffee bean for roasting. Our family just purchased another 5 pounds for more roasting. Plenty of flavor and mild on acidity when roasted to a dark brown bean and befor

Delicious!:  I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is traying different seasoning



In [47]:
results = search_reviews(df, "whole wheat pasta", n=3)

Tasty and Quick Pasta:  Barilla Whole Grain Fusilli with Vegetable Marinara is tasty and has an excellent chunky vegetable marinara.  I just wish there was more of it.  If you aren't starving or on a 

sooo good:  tastes so good. Worth the money. My boyfriend hates wheat pasta and LOVES this. cooks fast tastes great.I love this brand and started buying more of their pastas. Bulk is best.

Handy:  Love the idea of ready in a minute pasta and for that alone this product gets praise.  The pasta is whole grain so that's a big plus and it actually comes out al dente.  The vegetable marinara



&emsp;&emsp;我们来分析一下匹配结果，以第一个示例delicious beans（"美味豆类"）为例：
- 原文: "Good Buy: I liked the beans. They were vacuum sealed, plump and moist. Would recommend them for any use. I personally split and stuck them in some vodka to make vanilla extract. Yum!"
- 译文: "Good Buy：我喜欢这些豆子。它们是真空密封的，饱满而湿润。我会推荐它们用于任何用途。我个人将它们分开并放入一些伏特加中制作香草精。太美味了！"
---
- 原文: "Jamaican Blue beans: Excellent coffee bean for roasting. Our family just purchased another 5 pounds for more roasting. Plenty of flavor and mild on acidity when roasted to a dark brown bean and before..."
- 译文: "Jamaican Blue beans：非常适合烘焙的咖啡豆。我们家刚刚又购买了5磅用来烘焙。烘焙成深褐色豆子时，味道丰富，酸度温和……"
---
- 原文: "Delicious!: I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is trying different seasonings..."
- 译文: "美味极了！我喜欢这种白豆调味料，它让豆子味道浓郁，我太爱了，我婆婆以前不知道Zatarain's这个牌子，现在她开始尝试不同的调味品……"

&emsp;&emsp;第一条评论提到了用户对豆子的喜爱，提及它们的新鲜度以及多种用途，甚至用来制作香草精，体现了与"美味豆类"的相关性。第二条评论虽然是关于咖啡豆，但同样提到了豆子的优秀品质，符合了“美味”和“豆类”的搜索条件。第三条评论中，用户赞扬了某种调味品，使白豆更加美味，同样与"delicious beans"的搜索意图相匹配。

&emsp;&emsp;这样的匹配结果足以表明，**Embedding不仅能捕捉到关键词的表面匹配，还能理解语义的深层联系，精确地找到评论中的相关内容和用户情感，从而在文本搜索中达到更高的精确度。Embedding 这样的文本搜索方式以其快速、简便且高精度的特性，特别适用于构建问答系统、个性化推荐等领域。**

&emsp;&emsp;通过上面的六个典型案例实践来看，Embedding所蕴含的丰富语义信息使其在多个不同的应用场景中都能发挥不同的作用，**这种快速而精确的语义捕捉能力，使Embedding成为了连接文本数据与机器理解之间的关键桥梁。事实上，Embedding 不仅名气大，在企业级的业务中，无论是国外的 Facebook、Airbnb，还是我们国内的阿里、美团等大厂，我们都可以看到 Embedding 的成功应用**。比如Facebook的新闻推送算法考虑了用户与其朋友、公众号、团体等的互动，就是通过Embedding技术捕获这些复杂的关系并帮助优化用户在Feed中看到的内容；阿里的淘宝和天猫等平台使用Embedding技术来理解用户的购买行为和偏好，从而为他们推荐相关商品，**如果看了我们最近的SMP金融大模型竞赛方案直播的同学，应该也知道我们在对Question和金融文档匹配的过程中，也使用了Embedding。**

&emsp;&emsp;除此之外，**近期还有一个关键性的事件更是进一步的提升了Embedding的价值**，那就是在OpenAI在2023年11月06日发布会上，**新一代的gpt-4-turbo模型将文本输入限制扩展到了128k tokens，而gpt-3.5-turbo-1106也更新至16k tokens，并且降低了成本，这使得长文本处理变得更加经济实惠，而这种输入限制的扩大其实对于Embedding的应用产生了非常显著的潜在影响**，因为大模型和Embedding有很强的联系，如下图所示：

<div align=center><img src="https://snowball97.oss-cn-beijing.aliyuncs.com/images/202311121159902.jpg"></div>

&emsp;&emsp;大语言模型的核心工作是执行复杂的计算任务和处理庞大的数据集，以便准确预测文本序列中下一个单词的出现或补全给定的句子。这一过程可以分解为以下几个关键步骤：

- **文章**<br>始于输入文本，文本由一个或多个句子组成。这个文本是模型将要处理的内容；

- **Token**<br>将每个文本切分Tokens。Token可以是单词、单词的一部分，甚至是标点符号，将文本切分为tokens后，才能够输入不同模型进行识别、训练

- **Embedding**<br>每个Token都被转换为一种数值形式，可以捕获语义和句法信息之间的复杂关系；<br>

- **LLM**<br>将Embedding表示传递给语言模型。对于像GPT这样的模型，涉及到多层Transformer，它们是一种特定类型的神经网络架构。这些层被分为编码器和解码器；

    - **编码器**<br>每个编码器层处理输入的Embedding，以一种使模型能够开始理解不同Token之间关系的方式进行转换。它通过自注意机制等方式实现，使模型在考虑每个特定Token时能够权衡其他Token的重要性；

    - **解码器**<br>在编码之后，解码器层生成输出。它们接收处理过的Embedding，并逐步生成响应；

- **Output**<br>在文本通过整个模型之后生成最终输出。这个输出可以是文本形式，比如一个句子或段落，是模型处理的结果；

&emsp;&emsp;所以对于Embedding来说，在以下几方面也得到了非常大的增强：

1. **增强的上下文捕捉能力**：输入限制的提升使得模型能够处理更长的文本，这直接增强了Embedding的能力，使其能够包含更完整的上下文信息。长文本中的复杂语义和细节可以被准确地映射到向量空间中，为更精细的文本理解和分析提供了基础。

2. **更丰富的语义表示**：随着模型接受长度的增加，Embedding能够蕴含更多的语义层面信息。这使得无论是在文本相似度比较、情感分析还是主题建模等任务中，Embedding都能提供更丰富的特征表示，提高模型的性能和准确性。

3. **文本处理的连贯性增强**：对于长篇章的处理，之前可能需要将文本切割成小片段来适应模型的限制。现在，模型能够一次性处理长篇文章，这意味着整个文档的语义连贯性得以保持，提供了更为整体的文本理解视角。

4. **本地知识库的优化应用**：扩展的输入限制减少了对分割长文本的需求，使得模型能够直接应用于较大的数据集，这对于构建本地知识库尤为有利。本地化的Embedding可以更加高效，减少了对API调用的依赖，降低了延迟和成本。

5. **提升复杂应用的处理能力**：在需求复杂的NLP应用中，如对话系统、自动摘要生成等，长文本输入的支持为模型提供了处理更复杂语义和维护更长对话上下文的能力。


&emsp;&emsp;**随着大模型输入限制的扩展，可以预见，在当前的发展趋势下，处理大量文本数据的Embedding应用不仅将变得更加高效，而且其在各个领域的应用重要性也将日益增加**。当涉及到海量数据时，Embedding的应用将变得更加复杂，不再像我们之前探讨的六个案例那样直观简单。所以，**我们准备了四个针对海量文本处理的企业级Embedding案例及其解决策略**。包括**大规模文本数据的批量Embedding处理、自定义Embedding维度的优化策略、运用Embedding进行高效的海量文本匹配以及大数据集下的文本相似度分析聚类**。这四个案例直观的展示了大模型输入限制扩展对Embedding质量和效率的显著提升，并同时，大家也可以了解到，在企业级应用中，如何有效利用Embedding来提升业务价值。